# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
collections_list = db.list_collection_names()

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"hygiene_score": 20}

# Use count_documents to display the number of documents in the result
num_documents = collection.count_documents(query)
print(f"Number of documents with hygiene score 20: {num_documents}")
# Display the first document in the results using pprint
first_document = collection.find_one(query)
print("First document with hygiene score 20:")
pprint(first_document)

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(collection.find(query)))
# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")
# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
num_documents = collection.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {num_documents}")
# Display the first document in the results using pprint
first_document = collection.find_one(query)
print("First document in London with RatingValue >= 4:")
pprint(first_document)

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(collection.find(query)))
# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")
# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.5074
longitude = -0.1278 

query = {"RatingValue": 5,
         "geometry": {"$nearSphere": {"$geometry": {"type": "Point", "coordinates": [longitude, latitude]}, "$maxDistance": 0.01}}}
sort = [("HygieneScore", 1)]   

# Print the results
cursor = collection.find(query).sort(sort).limit(5)
for document in cursor:
    pprint(document)

In [ ]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(list(cursor))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results
pipeline = [
    {"$match": {"HygieneScore": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

result = list(collection.aggregate(pipeline))

for item in result:
    print(f"Local Authority: {item['_id']}, Number of establishments with hygiene score 0: {item['count']}")

print("First 10 results:")
pprint(result[:10])

In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
df = pd.DataFrame(result)

print(f"Number of rows in the DataFrame: {len(df)}")

print("First 10 rows of the DataFrame:")
print(df.head(10))